# Hand Cricket Game

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.models import load_model

In [3]:
# Loading the Finger_Prediction Model

model = load_model("Model.h5")
model.load_weights("Model_weights.h5")

In [4]:
# Function to process the image captured before sending it to the model for prediction

def imageProcessing(cut1):
    
    cut1 = cv2.cvtColor(cut1, cv2.COLOR_BGR2RGB)
    image_rgb = cv2.cvtColor(cut1, cv2.COLOR_BGR2RGB)
    
    mask = np.zeros(image_rgb.shape[:2], np.uint8)
    
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    
    rectangle = (50, 50, 200, 200)
    
    cv2.grabCut(image_rgb,
            mask,
            rectangle,
            bgdModel,
            fgdModel,
            5,
            cv2.GC_INIT_WITH_RECT)
    
    mask_2 = np.where((mask==2) | (mask==0), 0, 1).astype('uint8')
    
    image_rgb_nobg = image_rgb * mask_2[:, :, np.newaxis]
    
    img = cv2.cvtColor(image_rgb_nobg, cv2.COLOR_RGB2BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    return img


In [5]:
# Function to predict the count of Fingers and return the predicted value

def returnValues(cut1,cut2):
    img1 = np.array(cut1,dtype="float16")
    img2 = np.array(cut2,dtype="float16")
            
    X1 = np.argmax(model.predict(np.array([img1])),axis=1)
    X2 = np.argmax(model.predict(np.array([img2])),axis=1)
    
    return X1[0], X2[0]


In [6]:
# Function to capture hands of player1 and player2 and calculate the runs scored by player1

def P1_Video_Capture(player1, p1_runs, balls):

    cap =  cv2.VideoCapture(0)
    curr_ball = 1

    while True:
        retval,image = cap.read()

        if retval:
            x,y,w,h = (0,180,256,256)
            x2,y2,w2,h2 = (384,180,256,256)

            cut1 = image[y:y+h,x:x+w]
            cut2 = image[y2:y2+h2,x2:x2+w2]
            
            cut1 = imageProcessing(cut1)
            cut2 = imageProcessing(cut2)

            X1,X2 = returnValues(cut1,cut2)

            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),3)
            cv2.rectangle(image,(x2,y2),(x2+w2,y2+h2),(255,0,0),3)
            
            cv2.putText(image,("Ball {}".format(curr_ball)),(300,80),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
            
            cv2.putText(image,("P1: {}".format(X1)),(50,110),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
            cv2.putText(image,("P2: {}".format(X2)),(512,110),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
        
            cv2.putText(image,("{} Runs: {}".format(player1, p1_runs)),(50,140),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
            
            key = cv2.waitKey(1)
            
            if key == ord("p"):
                if curr_ball > balls or X1 == X2:
                    break
                
                if X1 == 0:
                    p1_runs += 6
                else:
                    p1_runs += X1
                
                curr_ball += 1

            cv2.imshow("Player 1 Video Capture",image)

            if key == ord("x"):
                break

    cap.release()
    cv2.destroyAllWindows()
    
    return p1_runs


In [7]:
# Function to capture hands of player1 and player2 and calculate the runs scored by player2

def P2_Video_Capture(player2, p2_runs, balls, target, player1):

    cap =  cv2.VideoCapture(0)
    curr_ball = 1

    while True:
        retval,image = cap.read()

        if retval:
            x,y,w,h = (0,180,256,256)
            x2,y2,w2,h2 = (384,180,256,256)

            cut1 = image[y:y+h,x:x+w]
            cut2 = image[y2:y2+h2,x2:x2+w2]
            
            cut1 = imageProcessing(cut1)
            cut2 = imageProcessing(cut2)

            X1,X2 = returnValues(cut1,cut2)

            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),3)
            cv2.rectangle(image,(x2,y2),(x2+w2,y2+h2),(255,0,0),3)
            
            cv2.putText(image,("Ball {}".format(curr_ball)),(300,80),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
            
            cv2.putText(image,("P1: {}".format(X1)),(50,110),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
            cv2.putText(image,("P2: {}".format(X2)),(512,110),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
        
            cv2.putText(image,("Target: {}".format(target)),(512,140),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
            cv2.putText(image,("{} need {} runs in {} balls to win".format(player2, (p1_runs-p2_runs+1), (balls-curr_ball+1))),(156,170),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
            
            cv2.putText(image,("{} Runs: {}".format(player2, p2_runs)),(50,140),cv2.FONT_HERSHEY_DUPLEX,0.50,(0,100,0),2)
            
            key = cv2.waitKey(1)
            
            if key == ord("p"):
                if curr_ball > balls or X1 == X2:
                    break
                
                if X2 == 0:
                    p2_runs += 6
                else:
                    p2_runs += X2
                
                curr_ball += 1
                
                if p2_runs>p1_runs:
                    break

            cv2.imshow("Player 2 Video Capture",image)

            if key == ord("x"):
                break

    cap.release()
    cv2.destroyAllWindows()
    
    Result(p1_runs, p2_runs, player1, player2)
    

In [8]:
# Function that receives the total runs scored by player1 and player2 and outputs the winner of the game 

def Result(p1_runs, p2_runs, player1, player2):

    cap =  cv2.VideoCapture(0)

    while True:
        retval,image = cap.read()

        if retval:
            
            cv2.putText(image,("Score:"),(15,80),cv2.FONT_HERSHEY_COMPLEX,0.75,(0,100,0),2)
            cv2.putText(image,("{} Runs: {}".format(player1, p1_runs)),(15,110),cv2.FONT_HERSHEY_COMPLEX,0.75,(0,100,0),2)
            cv2.putText(image,("{} Runs: {}".format(player2, p2_runs)),(15,140),cv2.FONT_HERSHEY_COMPLEX,0.75,(0,100,0),2)
            
            if p1_runs > p2_runs:
                cv2.putText(image,("{} Won by {} runs!!".format(player1, (p1_runs-p2_runs))),(15,170),cv2.FONT_HERSHEY_COMPLEX,0.75,(0,100,0),2)
            elif p2_runs > p1_runs:
                cv2.putText(image,("{} Won by {} runs!!".format(player2, (p2_runs-p1_runs))),(15,170),cv2.FONT_HERSHEY_COMPLEX,0.75,(0,100,0),2)
            else:
                cv2.putText(image,("Match Drawn!!"),(15,170),cv2.FONT_HERSHEY_COMPLEX,0.75,(0,100,0),2)

            cv2.imshow("Final Result",image)

            key = cv2.waitKey(1)

            if key == ord("x"):
                break

    cap.release()
    cv2.destroyAllWindows() 


In [11]:
balls = int(input("Enter the number of balls you want to play"))
player1 = input("Enter the name of Player 1: ")
player2 = input("Enter the name of Player 2: ")

p1_runs = P1_Video_Capture(player1, 0, balls)

target = p1_runs + 1

P2_Video_Capture(player2, 0, balls, target, player1)

Enter the number of balls you want to play 5
Enter the name of Player 1:  Pranshu
Enter the name of Player 2:  Apoorv
